approach as in subtaskA_1, adapted for dev data
& added Ann's code to save results in the correct format for submission

In [ ]:
# read tsv file
import csv
import sys
import pandas as pd
import ast

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [2]:
# Code von Wiebke

fileName = "subtask_a_dev.tsv"
fileDirectory = "AdMIRe Subtask A Dev/dev"

data = pd.read_csv(fileDirectory + "/" + fileName, sep='\t')

In [3]:
def sim_scores(current, sentences):
    # input = current line(example) & embeddings for sentence + captions
    
    scores = {}
    # keys = image names
    # values = scores

    embeddings = model.encode(sentences)
    similarities = model.similarity(embeddings[0], embeddings)
    # compares the embedding for the sentence including the compound 
    # with each of the embeddings, including itself and all the captions

    # [0][x] required because similarities tensor has additional layer
    score1 = similarities[0][1].item()
    scores[current["image1_name"]] = score1

    score2 = similarities[0][2].item()
    scores[current["image2_name"]] = score2

    score3 = similarities[0][3].item()
    scores[current["image3_name"]] = score3

    score4 = similarities[0][4].item()
    scores[current["image4_name"]] = score4

    score5 = similarities[0][5].item()
    scores[current["image5_name"]] = score5
    
    return scores

In [4]:
def rank_images(scores):
    ranking = []
    # scores = dictionary containing the cos similarity scores
    # comparing the sentence with the captions of the five images
    # keys = image names
    # values = scores

    for i in range(5):
        # find key which corresponds to the highest value
        m = max(scores, key=scores.get)
        # add the key (image name) to the ranking
        ranking.append(m)
        # delete the entry in the dictionary
        del scores[m]

    return ranking

In [ ]:
# compare embeddings for context sentence (including compound) with image captions

submission = []

for i in range(len(data)):

    current = data.iloc[i]
    sentences = [current["sentence"], 
                 current["image1_caption"],
                 current["image2_caption"],
                 current["image3_caption"],
                 current["image4_caption"],
                 current["image5_caption"]]

    scores = sim_scores(current, sentences)
    ranking = rank_images(scores)

    submission.append({
        "compound": current["compound"],
        "expected_order":ranking 
    })

# Save the submission data to a .tsv file
submission_df = pd.DataFrame(submission)
submission_df.to_csv("submission_EN.tsv", sep="\t", index=False)
print("File saved as submission_EN.tsv")

In [ ]:
# comparing embeddings for captions with that of compound (not whole sentence)

submission = []

for i in range(len(data)):

    current = data.iloc[i]
    sentences = [current["compound"], 
                 current["image1_caption"],
                 current["image2_caption"],
                 current["image3_caption"],
                 current["image4_caption"],
                 current["image5_caption"]]

    scores = sim_scores(current, sentences)
    ranking = rank_images(scores)

    submission.append({
        "compound": current["compound"],
        "expected_order":ranking 
    })

submission_df = pd.DataFrame(submission)
submission_df.to_csv("submission_EN.tsv", sep="\t", index=False)
print("File saved as submission_EN.tsv")

In [7]:
# combining "sentence" and "compound" embeddings (average)

def sim_scores_combined(current, sentences):
    # input = current line(example) & embeddings for sentence + captions
    
    scores = {}
    # keys = image names
    # values = scores

    embeddings = model.encode(sentences)

    # combining compound & sentence embeddings
    sent_comp = (embeddings[0] + embeddings[1]) / 2

    similarities = model.similarity(sent_comp, embeddings[2:])
    # compares the embedding for the sentence and compound combined
    # with each of the embeddings, including itself and all the captions

    # [0][x] required because similarities tensor has additional layer
    score1 = similarities[0][0].item()
    scores[current["image1_name"]] = score1

    score2 = similarities[0][1].item()
    scores[current["image2_name"]] = score2

    score3 = similarities[0][2].item()
    scores[current["image3_name"]] = score3

    score4 = similarities[0][3].item()
    scores[current["image4_name"]] = score4

    score5 = similarities[0][4].item()
    scores[current["image5_name"]] = score5
    
    return scores

In [ ]:
# comparing embeddings for captions with that of compound and sentence

submission = []

for i in range(len(data)):

    current = data.iloc[i]
    sentences = [current["compound"], 
                 current["sentence"], 
                 current["image1_caption"],
                 current["image2_caption"],
                 current["image3_caption"],
                 current["image4_caption"],
                 current["image5_caption"]]

    scores = sim_scores_combined(current, sentences)
    ranking = rank_images(scores)

    submission.append({
        "compound": current["compound"],
        "expected_order":ranking 
    })

submission_df = pd.DataFrame(submission)
submission_df.to_csv("submission_EN.tsv", sep="\t", index=False)
print("File saved as submission_EN.tsv")